In [32]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid
from sklearn import preprocessing, svm, metrics, tree, decomposition
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from time import time

from sklearn.metrics import log_loss

from sklearn.model_selection import StratifiedKFold

In [33]:
temp = os.path.expanduser('C:\\Users\\pmeji\\Documents\\ITAM2\\MineriaDatos2\\proyecto-final\\build\\temp\\')
results = os.path.expanduser('C:\\Users\\pmeji\\Documents\\ITAM2\\MineriaDatos2\\proyecto-final\\build\\results\\')

### Read data

In [7]:
addr= os.path.join(temp, "walmart_test_trip.csv")
df_test = pd.read_csv(addr)
df_test_sample =  df_test[:100]

addr= os.path.join(temp, "walmart_train_trip.csv")
df_train = pd.read_csv(addr)
df_train_sample =  df_train[:100]

## Diseño de prueba de modelo

In [113]:
def define_hyper_params():
    clfs = {
        'RF': RandomForestClassifier(n_estimators=10),
        'ET': ExtraTreesClassifier(n_estimators=10, criterion='entropy'),
        'AB': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=200),
        'LR': LogisticRegression(penalty='l2', C=1e5, solver='liblinear'),
        'SVM': svm.SVC(kernel='linear', probability=True, random_state=0),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10),
        'NB': GaussianNB(),
        'DT': DecisionTreeClassifier(),
        'SGD': SGDClassifier(loss="hinge", penalty="l2"),
        'KNN': KNeighborsClassifier(n_neighbors=1),
        'XGB': XGBClassifier()
            }

    grid = { 
        'RF':{'Classifier__n_estimators': [1,10,100,1000,10000], 'Classifier__max_depth': [1,5,10,20,50,100], 
              'Classifier__max_features': ['sqrt','log2'],'Classifier__min_samples_split': [2,5,10]},
        'LR': { 'Classifier__penalty': ['l1','l2'], 'Classifier__C': [0.00001,0.0001,0.001,0.01,0.1,1,10]},
        'SGD': { 'Classifier__loss': ['hinge','log','perceptron'], 'Classifier__penalty': ['l2','l1','elasticnet']},
        'ET': { 'Classifier__n_estimators': [1,10,100,1000,10000], 'Classifier__criterion' : ['gini', 'entropy'] ,
               'Classifier__max_depth': [1,5,10,20,50,100], 'Classifier__max_features': ['sqrt','log2'],
               'Classifier__min_samples_split': [2,5,10]},
        'AB': { 'Classifier__algorithm': ['SAMME', 'SAMME.R'], 'Classifier__n_estimators': [1,10,100,1000,10000]},
        'GB': {'Classifier__n_estimators': [1,10,100,1000,10000], 'Classifier__learning_rate' : [0.001,0.01,0.05,0.1,0.5],
               'Classifier__subsample' : [0.1,0.5,1.0], 'Classifier__max_depth': [1,3,5,10,20,50,100]},
        'NB' : {},
        'DT': {'Classifier__criterion': ['gini', 'entropy'], 'Classifier__max_depth': [1,5,10,20,50,100], 
               'Classifier__max_features': ['sqrt','log2'],'Classifier__min_samples_split': [2,5,10]},
        'SVM' :{'Classifier__C' :[0.00001,0.0001,0.001,0.01,0.1,1,10],'Classifier__kernel':['linear', 'rbf']},
        'KNN' :{'Classifier__n_neighbors': [1,5,10,25,50,100],'Classifier__weights': ['uniform','distance'],
                'Classifier__algorithm': ['auto','ball_tree','kd_tree']},
        #'KNN' :{},
        #'LR':{},
        'XGB':  {'Classifier__learning_rate': [0.2, 0.3, 0.5], 'Classifier__max_depth': [6, 9, 12],
                    'Classifier__subsample': [0.8, 1.0],  'Classifier__colsample_bytree': [0.8, 1.0]} 
           }

    return clfs, grid 
                     


In [126]:
from sklearn.metrics import log_loss, make_scorer


def magic_loop(models_to_run, clfs, grid, pipeline, df_train, df_test, search =1):
    type_labs = list(set(df_train.triptype.ravel()))
    y = df_train.triptype.ravel()
    X = df_train.drop(['triptype'], axis=1)
    
    results_logloss = {}
    
    for index, clf in enumerate([clfs[x] for x in models_to_run]):
        model_name = models_to_run[index]
        print(model_name)
        parameter_values = grid[models_to_run[index]]
        parameter_values['PCA__n_components'] = [2,4] # [2,4,6,8]
        try:
            pipeline.set_params(Classifier = clf)
            if(search):
                type_labs = list(set(df_train.triptype.ravel()))
                LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True,labels=type_labs)
                
                gs = GridSearchCV(pipeline, parameter_values, cv=StratifiedKFold(2), 
                                  n_jobs=-1, scoring='neg_log_loss', verbose= 1)
            else:
                gs = RandomizedSearchCV(pipeline, parameter_values, cv=StratifiedKFold(2),
                                         n_jobs=-1, scorer=sklearn.metrics.log_loss(labels=type_labs))
            start = time()

            gs.fit(X, y)
            y_pred = gs.predict_proba(df_test)
            
            nom_csv = "cv_"+ str(model_name) +".csv"
            addr= os.path.join(results, nom_csv)
            pd.DataFrame(gs.cv_results_).to_csv(addr)


            print("GridSearch time: " + (str)(time() - start))

        except IndexError as e:
            print('Error:', e)
            continue

    return results_logloss

In [ ]:
pipeline = Pipeline([
    ('VarianceThreshold', VarianceThreshold()),
    ('Normalizer', StandardScaler()),
    ('Classifier', RandomForestClassifier(n_estimators=10, n_jobs=-1))
    ])

models = ['KNN']

df_train_sample =  df_train
clfs, grid = define_hyper_params()
results_S = magic_loop( models, clfs, grid, pipeline, df_train_sample)



In [ ]:
pipeline = Pipeline([
    ('VarianceThreshold', VarianceThreshold()),
    ('Normalizer', StandardScaler()),
    ('PCA', PCA()),
    ('Classifier', RandomForestClassifier(n_estimators=10, n_jobs=-1))
    ])

models = ['LR', 'RF', 'KNN']

df_train_sample =  df_train
clfs, grid = define_hyper_params()
results_S = magic_loop( models, clfs, grid, pipeline, df_train, df_test)

results_S

LR
Fitting 2 folds for each of 28 candidates, totalling 56 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.3s
[Parallel(n_jobs=-1)]: Done  56 out of  56 | elapsed:  1.1min finished
C:\Users\pmeji\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearch time: 78.89238667488098
RF
Fitting 2 folds for each of 360 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 104.3min


In [76]:
y = df_train.triptype.ravel()
X = df_train.drop(['triptype'], axis=1)

X_test = df_test

clf = LogisticRegression(penalty='l2', C=1e5, solver='liblinear')
clf.fit(X,y)

y_pred = clf.predict_proba(X_test)

C:\Users\pmeji\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [82]:
test_predictions.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,0,0.012945,0.002348,0.053498,0.016981,0.072491,0.062615,0.048586,0.006056,0.000464,...,0.021763,0.014799,0.016785,0.134146,0.129685,0.006680,0.019215,0.033020,0.029003,0.060409
1,1,0.014420,0.002766,0.054421,0.018362,0.072086,0.063492,0.050149,0.006427,0.000657,...,0.021321,0.011707,0.015565,0.128580,0.122084,0.006521,0.019089,0.033002,0.027847,0.060862
2,2,0.032505,0.007846,0.064205,0.036413,0.071720,0.075172,0.068538,0.007934,0.000844,...,0.017367,0.008464,0.011048,0.095142,0.075289,0.005449,0.014434,0.035715,0.020322,0.077159
3,3,0.035537,0.008708,0.064231,0.036450,0.071635,0.075258,0.069000,0.007915,0.000714,...,0.016526,0.008147,0.011144,0.094618,0.074235,0.005186,0.014117,0.035367,0.019520,0.073194
4,4,0.032665,0.007874,0.064102,0.036378,0.071659,0.075007,0.068542,0.007952,0.000858,...,0.016484,0.008662,0.011299,0.094928,0.075041,0.005518,0.014544,0.035569,0.020280,0.077069


In [108]:
test_predictions = pd.DataFrame(y_pred)
test_indexes = df_test.visitnumber 
test_predictions.insert(0, 'VisitNumber', test_indexes)
test_predictions.columns = ['VisitNumber', 'TripType_3','TripType_4','TripType_5','TripType_6','TripType_7',\
'TripType_8','TripType_9','TripType_12','TripType_14','TripType_15','TripType_18',\
'TripType_19','TripType_20','TripType_21','TripType_22','TripType_23','TripType_24',\
'TripType_25','TripType_26','TripType_27','TripType_28','TripType_29','TripType_30',\
'TripType_31','TripType_32','TripType_33','TripType_34','TripType_35','TripType_36',\
'TripType_37','TripType_38','TripType_39','TripType_40','TripType_41','TripType_42',\
'TripType_43','TripType_44','TripType_999']

addr= os.path.join(results, "./submissions/LR_16.csv")
test_predictions.to_csv(addr, index=False)

In [104]:
test_predictions[:-5]

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1.0,1.294458e-02,2.348261e-03,5.349795e-02,1.698110e-02,7.249121e-02,6.261535e-02,4.858591e-02,6.055906e-03,4.639242e-04,...,0.021763,0.014799,0.016785,0.134146,0.129685,0.006680,0.019215,3.301976e-02,0.029003,6.040874e-02
1,2.0,1.441968e-02,2.765756e-03,5.442071e-02,1.836193e-02,7.208578e-02,6.349219e-02,5.014894e-02,6.427093e-03,6.571939e-04,...,0.021321,0.011707,0.015565,0.128580,0.122084,0.006521,0.019089,3.300198e-02,0.027847,6.086227e-02
2,3.0,3.250462e-02,7.846486e-03,6.420476e-02,3.641344e-02,7.171996e-02,7.517239e-02,6.853767e-02,7.934113e-03,8.444970e-04,...,0.017367,0.008464,0.011048,0.095142,0.075289,0.005449,0.014434,3.571535e-02,0.020322,7.715897e-02
3,4.0,3.553717e-02,8.708009e-03,6.423148e-02,3.644986e-02,7.163505e-02,7.525783e-02,6.899997e-02,7.914899e-03,7.137106e-04,...,0.016526,0.008147,0.011144,0.094618,0.074235,0.005186,0.014117,3.536706e-02,0.019520,7.319449e-02
4,6.0,3.266545e-02,7.874061e-03,6.410195e-02,3.637794e-02,7.165876e-02,7.500735e-02,6.854228e-02,7.951696e-03,8.583134e-04,...,0.016484,0.008662,0.011299,0.094928,0.075041,0.005518,0.014544,3.556872e-02,0.020280,7.706918e-02
5,13.0,2.607624e-02,5.742889e-03,6.176444e-02,2.921423e-02,7.300879e-02,7.239870e-02,6.300425e-02,7.393601e-03,6.194822e-04,...,0.018179,0.009934,0.012716,0.107411,0.091206,0.005671,0.015729,3.545679e-02,0.022692,7.019720e-02
6,14.0,8.100049e-04,1.021433e-04,2.014314e-02,1.824454e-03,4.499011e-02,2.181826e-02,1.120368e-02,2.538367e-03,2.705716e-04,...,0.028754,0.027014,0.025558,0.197858,0.271896,0.008412,0.028291,1.826229e-02,0.047958,2.051363e-02
7,16.0,1.046682e-03,1.488892e-04,2.206052e-02,2.250137e-03,4.674382e-02,2.374361e-02,1.265372e-02,3.015308e-03,5.655191e-04,...,0.028558,0.018267,0.025501,0.187921,0.252431,0.008374,0.028849,1.917644e-02,0.046057,2.199445e-02
8,18.0,4.284634e-03,6.624693e-04,3.828873e-02,7.093340e-03,6.360481e-02,4.371150e-02,2.847496e-02,4.447907e-03,4.073707e-04,...,0.025688,0.019687,0.020649,0.166150,0.189274,0.007684,0.023784,2.733725e-02,0.037698,4.159535e-02
9,21.0,3.802461e-03,5.762739e-04,3.716377e-02,6.454096e-03,6.301390e-02,4.236620e-02,2.709493e-02,4.115897e-03,2.897473e-04,...,0.026061,0.023730,0.020459,0.171780,0.198593,0.007760,0.023775,2.701852e-02,0.038893,4.063388e-02


In [107]:
int(df_test.visitnumber)

TypeError: cannot convert the series to <class 'int'>

0             1.0
1             2.0
2             3.0
3             4.0
4             6.0
5            13.0
6            14.0
7            16.0
8            18.0
9            21.0
10           22.0
11           24.0
12           27.0
13           34.0
14           35.0
15           36.0
16           37.0
17           38.0
18           39.0
19           44.0
20           46.0
21           48.0
22           52.0
23           58.0
24           59.0
25           60.0
26           62.0
27           64.0
28           65.0
29           66.0
           ...   
95644    191299.0
95645    191300.0
95646    191302.0
95647    191303.0
95648    191306.0
95649    191307.0
95650    191308.0
95651    191309.0
95652    191310.0
95653    191314.0
95654    191315.0
95655    191316.0
95656    191317.0
95657    191319.0
95658    191320.0
95659    191321.0
95660    191323.0
95661    191325.0
95662    191327.0
95663    191328.0
95664    191330.0
95665    191332.0
95666    191333.0
95667    191334.0
95668    1